In [55]:
from scrape_school_url import *
import pandas as pd
from bs4 import BeautifulSoup
import re
from os import path, makedirs

In [51]:
date.today().strftime('%Y-%m')

'2022-09'

In [80]:
ROOT_DIR = 'html/'+date.today().strftime('%Y-%m')
BASE_URL = 'https://data.bopp-obec.info/emis'
SCRAPING_URLS = {
  'general': BASE_URL+'/schooldata-view.php',
  'student': BASE_URL+'/schooldata-view_student.php',
  'staff': BASE_URL+'/schooldata-view.php',
  'computer_internet': BASE_URL+'/schooldata-view_com-internet.php',
  'building': BASE_URL+'/schooldata-view_bobec.php',
  'durable_goods': BASE_URL+'/schooldata-view_mobec.php?'
}
SCHOOL_DATA_FEILDS = list(SCRAPING_URLS)

In [90]:
def is_path_existed(file_path):
  return path.exists(file_path)

In [91]:
scrape_dirs = {}
for d in SCRAPING_URLS:
  scrape_dir = ROOT_DIR+'/'+d
  if not is_path_existed(scrape_dir):
    makedirs(scrape_dir)
  scrape_dirs[d] = scrape_dir

In [61]:
sdi = SchoolDataIndex()

In [93]:
def load_school_data(school_id: str):
  """
  load the school data by 10 digits school id. If they exis on the local
  then it will load from the local storage, or else it will scrape to
  the local storage

  params:
    - school_id 10 digits school id
  """
  data = {}
  for feild in SCHOOL_DATA_FEILDS:
    html_file_path = f'{scrape_dirs[feild]}/{school_id}.html'
    if is_path_existed(html_file_path):
      pass
    else:
      url = f'{SCRAPING_URLS[feild]}?School_ID={school_id}'
      scrape_url(url, html_file_path)
    data[feild] = html_file_path
  return data

load_school_data('1012230159')

{'general': 'html/2022-09/general/1012230159.html',
 'student': 'html/2022-09/student/1012230159.html',
 'staff': 'html/2022-09/staff/1012230159.html',
 'computer_internet': 'html/2022-09/computer_internet/1012230159.html',
 'building': 'html/2022-09/building/1012230159.html',
 'durable_goods': 'html/2022-09/durable_goods/1012230159.html'}

In [ ]:
def find_lat_lng(html_content):
  school_lat_lng_re = f'LatLng\((\d+\.\d+,\s*\d+\.\d+)\)'
  re.findall(school_lat_lng_re, html_content)

In [97]:
# about_school = []
# for table in soup.find('table').find_all('table'):
#   if 'width' in table.attrs.keys() and table.attrs['width'] == '521':
#     for table_row in table.find_all('tr'):
#       cells = table_row.find_all('td')
#       if len(cells) == 2:
#         key_name = cells[0].text.strip()
#         value = re.sub('\s+',' ', cells[1].text.strip())

#         if cells[1].find_all('a'):
#           value = [{'text': a_tag.text,'href': a_tag.attrs['href']} for a_tag in cells[1].find_all('a')]
        
#         about_school.append({
#             'key': key_name,
#             'value': value
#           })
#       else:
#         about_school.append({'value': cells[0].text.strip()})

# about_school

[{'key': 'รหัสโรงเรียน 10 หลัก :', 'value': '1010720019'},
 {'key': 'รหัส Smis 8 หลัก :', 'value': '10010001'},
 {'key': 'รหัส Obec 6 หลัก :', 'value': '720019'},
 {'key': 'ชื่อสถานศึกษา(ไทย) :', 'value': 'วัดชนะสงคราม'},
 {'key': 'ชื่อสถานศึกษา(อังกฤษ) :', 'value': 'Watchanasongkram'},
 {'key': 'ที่อยู่ :', 'value': 'หมู่ที่ - บ้านบางลำภู เลขที่ 77/1'},
 {'key': 'ตำบล :', 'value': 'ชนะสงคราม'},
 {'key': 'อำเภอ :', 'value': 'เขตพระนคร'},
 {'key': 'จังหวัด :', 'value': 'กรุงเทพมหานคร'},
 {'key': 'รหัสไปรษณีย์ :', 'value': '10200'},
 {'key': 'โทรศัพท์ :', 'value': '020436548'},
 {'key': 'โทรสาร :', 'value': '022820168'},
 {'key': 'ระดับที่เปิดสอน :', 'value': 'อนุบาล-ประถมศึกษา'},
 {'key': 'วัน-เดือน-ปี ก่อตั้ง :', 'value': ''},
 {'key': 'อีเมล์ :', 'value': ''},
 {'key': 'เว็บไซต์ :',
  'value': [{'text': 'www.watchanasongkram.ac.th/',
    'href': 'https://www.watchanasongkram.ac.th/'},
   {'text': 'เว็บไซต์โรงเรียน (สารสนเทศ)',
    'href': '../web/?School_ID=1010720019'}]},
 {'key': 'เ